# Network Intrusion Detection based on Machine Learning

In [154]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

# Reading the Dataset

In [155]:
data = pd.read_csv("network_intrusion_dataset.csv")

$head()$ will display the top 5 observations of the dataset

In [156]:
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,tcp,private,REJ,0.0,0.0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,anomaly
1,2,tcp,ftp_data,SF,12983.0,0.0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
2,0,icmp,eco_i,SF,20.0,0.0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,anomaly
3,0,tcp,http,SF,267.0,14515.0,0,0,0,0,...,255,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal
4,0,tcp,smtp,SF,1022.0,387.0,0,0,0,0,...,28,0.11,0.72,0.00,0.00,0.00,0.0,0.72,0.04,normal


### Analyzing the data

$info()$ helps to understand the data type and information about data, including the number of records in each column, data having null or not null, Data type, the memory usage of the dataset.

We can find that the network intrusion dataset has 40 columns, among them, "protocol_type", "service", "flag", and "class" columns are object type or categorical features (Not numerical)

In [157]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16128 entries, 0 to 16127
Data columns (total 40 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     16128 non-null  int64  
 1   protocol_type                16128 non-null  object 
 2   service                      16128 non-null  object 
 3   flag                         16128 non-null  object 
 4   src_bytes                    16122 non-null  float64
 5   dst_bytes                    16122 non-null  float64
 6   land                         16128 non-null  int64  
 7   wrong_fragment               16128 non-null  int64  
 8   urgent                       16128 non-null  int64  
 9   hot                          16128 non-null  int64  
 10  num_failed_logins            16128 non-null  int64  
 11  logged_in                    16128 non-null  int64  
 12  num_compromised              16128 non-null  int64  
 13  root_shell      

Some machine learning algorithms can handle categorical features directly without requiring them to be converted to numerical values. For example, Decision Trees and Random Forests can handle categorical variables naturally by splitting on different categories.

However, many other algorithms, such as logistic regression, support vector machines, and neural networks, require numerical input. In such cases, converting categorical features to numerical values becomes necessary. This process is called "encoding" categorical variables.

Since we will work with all classification models, we will use LabelEncoder() to convert the categorical features to numerical values.

In [158]:
label_encoder = LabelEncoder()
data['protocol_type'] = label_encoder.fit_transform(data['protocol_type'])
data['service'] = label_encoder.fit_transform(data['service'])
data['flag'] = label_encoder.fit_transform(data['flag'])
data['class'] = label_encoder.fit_transform(data['class'])

In [159]:
data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,45,1,0.0,0.0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,0
1,2,1,19,9,12983.0,0.0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,1
2,0,0,13,9,20.0,0.0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,0
3,0,1,22,9,267.0,14515.0,0,0,0,0,...,255,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,1
4,0,1,49,9,1022.0,387.0,0,0,0,0,...,28,0.11,0.72,0.00,0.00,0.00,0.0,0.72,0.04,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16123,0,1,25,1,0.0,0.0,0,0,0,0,...,18,0.07,0.05,0.00,0.00,0.00,0.0,1.00,1.00,0
16124,0,0,14,9,1032.0,0.0,0,0,0,0,...,255,1.00,0.00,1.00,0.00,0.00,0.0,0.00,0.00,0
16125,0,1,49,9,794.0,333.0,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,1
16126,0,1,22,9,317.0,938.0,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,1


### Checking class distribution

We need to check for class imbalance in the dataset, which means, we need to check if the ratio of normal data (now converted to 0) and anomalous data (now converted to 1) is equal or not. 

In [160]:
label_counts = data['class'].value_counts()

# Print the counts
print("Number of data labeled as 'anomaly':", label_counts[1])
print("Number of data labeled as 'normal':", label_counts[0])

Number of data labeled as 'anomaly': 9711
Number of data labeled as 'normal': 6417


As we can see, the dataset has more normal data than anomalous data, resulting in a class imbalance

### Checking for missing values

$isnull()$ is used to identify null/ missing values in the data
$isnull().sum()$ will show us the total number of missing records in each column. For example, "src_bytes" and "dst_bytes" each have 6 values missing.

In [161]:
data.isnull().sum() 

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      6
dst_bytes                      6
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
num_root                       0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          7
srv_count                      7
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_host_rate             0
dst_host_count                 0
dst_host_s

We are filling the missing values with the mean of all values in that column

In [162]:
data.fillna(data.mean(), inplace=True)

Now our dataset has only numerical values. This is easier to analyze using the machine learning classifiers

In [163]:
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,class
0,0,1,45,1,0.0,0.0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,0
1,2,1,19,9,12983.0,0.0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,1
2,0,0,13,9,20.0,0.0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,0
3,0,1,22,9,267.0,14515.0,0,0,0,0,...,255,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,1
4,0,1,49,9,1022.0,387.0,0,0,0,0,...,28,0.11,0.72,0.00,0.00,0.00,0.0,0.72,0.04,1


Now we will split the independent variables or features from the target variables or labels. In other words, we will keep the features in $X$ variable the labels in $y$ variable.

In [164]:
X = data.drop(columns=['class'])
y = data['class'] 

We will utilize stratified k-fold to maintain a uniform class distribution as the original dataset. This ensures that each fold is representative of the overall dataset, providing a more reliable estimate of the model's performance, particularly for classification tasks. Stratified k-fold is particularly useful when there is class imbalance in the dataset.

In [165]:
skf = StratifiedKFold(n_splits = 5)

The numerical values of the features are still unscaled and widely varied. Therefore, we need to scale the values. Scaling ensures that all features contribute equally to the model fitting process by bringing them to a similar scale. Without scaling, features with larger magnitudes may dominate those with smaller magnitudes, leading to biased model training.

In [166]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)
X_scaled

array([[0.00000000e+00, 5.00000000e-01, 7.14285714e-01, ...,
        0.00000000e+00, 1.00000000e+00, 1.00000000e+00],
       [3.67302713e-05, 5.00000000e-01, 3.01587302e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.06349206e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 5.00000000e-01, 7.77777778e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 5.00000000e-01, 3.49206349e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.00000000e+00, 1.74603175e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

# Logistic Regression

In [167]:
from sklearn.linear_model import LogisticRegression

In [168]:
lr = LogisticRegression()
true_labels = []
pred_labels = []
accuracies = []

In [169]:
for train_index, test_index in skf.split(X_scaled, y):
    
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    lr.fit(X_train, y_train)
    prediction_lr = lr.predict(X_test)
    
    true_labels.extend(y_test)
    pred_labels.extend(prediction_lr)
    
    accuracy = accuracy_score(y_test, prediction_lr)
    accuracies.append(accuracy)

C:\Users\mehra\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\mehra\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [170]:
print('5-fold Accuracies', accuracies)
print('Mean Accuracy', np.mean(accuracies))

5-fold Accuracies [0.9228146311221327, 0.9175449473031618, 0.9187848729076256, 0.9147286821705426, 0.9196899224806202]
Mean Accuracy 0.9187126111968166


In [171]:
print('Accuracy of Logistic Regression classifier: {:.4f}'.format(accuracy_score(true_labels, pred_labels)))
A = confusion_matrix(true_labels, pred_labels)
print('Total testing data :', A[0][0] + A[0][1] + A[1][0] + A[1][1])
print('True Positive: ', A[0][0])
print('True Negative: ', A[1][1])
print('False Positive: ', A[0][1])
print('Flase Negative: ', A[1][0])
print(classification_report(true_labels, pred_labels, digits=4))

Accuracy of Logistic Regression classifier: 0.9187
Total testing data : 16128
True Positive:  5502
True Negative:  9315
False Positive:  915
Flase Negative:  396
              precision    recall  f1-score   support

           0     0.9329    0.8574    0.8935      6417
           1     0.9106    0.9592    0.9343      9711

    accuracy                         0.9187     16128
   macro avg     0.9217    0.9083    0.9139     16128
weighted avg     0.9194    0.9187    0.9181     16128



# Random Forest

In [172]:
from sklearn.ensemble import RandomForestClassifier

In [173]:
rf = RandomForestClassifier()
true_labels = []
pred_labels = []
accuracies = []

In [174]:
for train_index, test_index in skf.split(X_scaled, y):
    
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    rf.fit(X_train, y_train)
    prediction_rf = rf.predict(X_test)
    
    true_labels.extend(y_test)
    pred_labels.extend(prediction_rf)
    
    accuracy = accuracy_score(y_test, prediction_rf)
    accuracies.append(accuracy)

In [175]:
print('5-fold Accuracies', accuracies)
print('Mean Accuracy', np.mean(accuracies))

5-fold Accuracies [0.9888406695598264, 0.9885306881587105, 0.9851208927464352, 0.9872868217054264, 0.9872868217054264]
Mean Accuracy 0.9874131787751649


In [176]:
print('Accuracy of Random Forest classifier: {:.4f}'.format(accuracy_score(true_labels, pred_labels)))
B = confusion_matrix(true_labels, pred_labels)
print('Total testing data :', B[0][0] + B[0][1] + B[1][0] + B[1][1])
print('True Positive: ', B[0][0])
print('True Negative: ', B[1][1])
print('False Positive: ', B[0][1])
print('Flase Negative: ', B[1][0])
print(classification_report(true_labels, pred_labels, digits=4))

Accuracy of Random Forest classifier: 0.9874
Total testing data : 16128
True Positive:  6300
True Negative:  9625
False Positive:  117
Flase Negative:  86
              precision    recall  f1-score   support

           0     0.9865    0.9818    0.9841      6417
           1     0.9880    0.9911    0.9896      9711

    accuracy                         0.9874     16128
   macro avg     0.9873    0.9865    0.9869     16128
weighted avg     0.9874    0.9874    0.9874     16128



# XGBoost

In [177]:
import xgboost
from xgboost import XGBClassifier

In [178]:
xgb = XGBClassifier() 
true_labels = []
pred_labels = []
accuracies = []

In [179]:
for train_index, test_index in skf.split(X_scaled, y):
    
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    xgb.fit(X_train, y_train)
    prediction_xgb = xgb.predict(X_test)
    
    true_labels.extend(y_test)
    pred_labels.extend(prediction_xgb)
    
    accuracy = accuracy_score(y_test, prediction_xgb)
    accuracies.append(accuracy)

C:\Users\mehra\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:42:13] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\mehra\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:42:14] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\mehra\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:42:16] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\mehra\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:42:17] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\mehra\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:42:19] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [180]:
print('5-fold Accuracies', accuracies)
print('Mean Accuracy', np.mean(accuracies))

5-fold Accuracies [0.9913205207687539, 0.9903905765654061, 0.9882207067575945, 0.9882170542635659, 0.9875968992248062]
Mean Accuracy 0.9891491515160254


In [181]:
print('Accuracy of Random Forest classifier: {:.4f}'.format(accuracy_score(true_labels, pred_labels)))
B = confusion_matrix(true_labels, pred_labels)
print('Total testing data :', B[0][0] + B[0][1] + B[1][0] + B[1][1])
print('True Positive: ', B[0][0])
print('True Negative: ', B[1][1])
print('False Positive: ', B[0][1])
print('Flase Negative: ', B[1][0])
print(classification_report(true_labels, pred_labels, digits=4))

Accuracy of Random Forest classifier: 0.9891
Total testing data : 16128
True Positive:  6312
True Negative:  9641
False Positive:  105
Flase Negative:  70
              precision    recall  f1-score   support

           0     0.9890    0.9836    0.9863      6417
           1     0.9892    0.9928    0.9910      9711

    accuracy                         0.9891     16128
   macro avg     0.9891    0.9882    0.9887     16128
weighted avg     0.9891    0.9891    0.9891     16128



As we can see, XGBoost is already performing very well, returning F1 score of 98.91%